# Install required packages
### We need hl7 library to parse HL7 segments
### Using conda to install helps to install all the dependencies 

In [4]:
# Install libraries 
! pip install -U hl7
%conda install openai==1.12.0 
 
# ! pip install -U hl7
# ! pip install --force-reinstall typing-extensions==4.5
# ! pip install --force-reinstall openai==1.8 --upgrade

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 21, Finished, Available)

Solving environment: - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.



In [75]:
# Import libraries 
import hl7
import pandas as pd
from delta.tables import *



StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 93, Finished, Available)

# Get Openai Key from Keyvault

In [ ]:
from notebookutils.mssparkutils.credentials import getSecret

KEYVAULT_ENDPOINT = "https://keyvaultgrfabric.vault.azure.net/"

OPENAI_API_KEY = getSecret(KEYVAULT_ENDPOINT, "AZURE-OPENAI-KEY")

In [7]:
# Path the hl7 file parse
document_path = f"abfss://799a0ea8-6c83-4265-8634-008dccddfe84@onelake.dfs.fabric.microsoft.com/fbcc24f4-768f-460e-88ac-bb668d740476/Files/raw_data/hl7messagePlaintext.txt"

df = spark.read.format("text").load(document_path)
# .select("_metadata.file_name", "content").limit(10).cache()

# display(df)


StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 25, Finished, Available)

In [8]:
# Collect all rows to a string
hl7_lines = df.collect()

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 26, Finished, Available)

In [9]:
# If you want the HL7 message as a single string variable
hl7_message = "\n".join([row.value for row in hl7_lines])

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 27, Finished, Available)

In [10]:
# Append \r at end of text for proper parsing
hl7_message=hl7_message.replace("\n", "\r") + "\r"

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 28, Finished, Available)

In [11]:
h = hl7.parse(hl7_message)

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 29, Finished, Available)

In [12]:
h['OBX'][0][3][0][1]

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 30, Finished, Available)

['Clinical Notes']

In [16]:
h['OBX'][0][5]

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 34, Finished, Available)

['Patient presents with a three-day history of nonspecific abdominal pain, moderate in intensity, localized to the lower abdomen. No associated symptoms of fever, vomiting, or diarrhea. Past medical history is significant for similar episodes in the past year, evaluated with no specific diagnosis. Physical examination reveals mild tenderness in the lower abdominal quadrants, no rebound or guarding. Plan: Recommend abdominal ultrasound to rule out gallstones. Further evaluation with gastroenterology may be needed based on ultrasound results.']

In [14]:
h['OBX'][0]

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 32, Finished, Available)

[['OBX'],
 ['1'],
 ['TX'],
 [[['1000.1'], ['Clinical Notes'], ['L']]],
 [''],
 ['Patient presents with a three-day history of nonspecific abdominal pain, moderate in intensity, localized to the lower abdomen. No associated symptoms of fever, vomiting, or diarrhea. Past medical history is significant for similar episodes in the past year, evaluated with no specific diagnosis. Physical examination reveals mild tenderness in the lower abdominal quadrants, no rebound or guarding. Plan: Recommend abdominal ultrasound to rule out gallstones. Further evaluation with gastroenterology may be needed based on ultrasound results.'],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['F'],
 [''],
 [''],
 ['202402291200']]

# OPEN AI Integration

In [20]:
from openai import AzureOpenAI

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 38, Finished, Available)

In [21]:
azure_client = AzureOpenAI(
  azure_endpoint = 'https://polite-ground-030dc3103.4.azurestaticapps.net/api/v1', 
  api_key='3fa53477-34bd-42fc-bbe5-d562d0a95e98',  
  api_version="2023-09-01-preview"
)

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 39, Finished, Available)

In [40]:
#  Define Methods for Translation and Summarization and create vector embeddings to chat with the data


def translate_to_english_with_gpt4(text):
    response = azure_client.chat.completions.create(
        model="gpt-35-turbo-16k",  # Specify GPT-4's engine here
        messages=[
            {"role": "system", "content": "You are a language translation expert."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=1024,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return  response.choices[0].message.content.strip()


def summarize(text):
    response = azure_client.chat.completions.create(
        model="gpt-35-turbo-16k",  # Specify GPT-4's engine here
        messages=[
            {"role": "system", "content": "You are a language translation expert. Brief Summarize and list all Medication he/she took in order."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=1024,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return  response.choices[0].message.content.strip()


# Text in an unknown language to be translated
# unknown_language_text = "El paciente presenta un historial de tres días de dolor abdominal no específico, de intensidad moderada, localizado en el abdomen inferior. No hay síntomas asociados de fiebre, vómitos o diarrea. El historial médico pasado es significativo por episodios similares en el último año, evaluados sin un diagnóstico específico. El examen físico revela una leve sensibilidad en los cuadrantes abdominales inferiores, sin rebote ni defensa. Plan: Se recomienda una ecografía abdominal para descartar cálculos biliares. Puede ser necesaria una evaluación adicional con gastroenterología basada en los resultados de la ecografía."

prompt=f"Translate the following text to English:\n\n{h['OBX'][0][5]}"
# Translate the text using GPT-4
english_translation = translate_to_english_with_gpt4(prompt)

# Summarize the notes
summarized = summarize(english_translation)

# Create embeddings on the summarized notes
embedding_df = azure_client.embeddings.create(model="text-embedding-ada-002",input=summarized)
embedding_df_value = embedding_df.data[0].embedding
print("\nOrignal text:", english_translation)
print("\n\nSummarized text:", summarized)
print("\n\nSummarized text embeddings: ", embedding_df.data[0].embedding)

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 58, Finished, Available)


Orignal text: The patient is experiencing abdominal pain that has been going on for three days. The pain is not specific and is of moderate intensity, mainly felt in the lower abdomen. There are no other symptoms such as fever, vomiting, or diarrhea. The patient has previously experienced similar episodes in the past year, but no specific diagnosis was made. During the physical examination, mild tenderness was found in the lower abdominal quadrants, but there were no signs of rebound or guarding. The plan is to suggest an abdominal ultrasound to check for gallstones. Depending on the results of the ultrasound, further evaluation with a gastroenterologist may be necessary.


Summarized text: The patient has been experiencing abdominal pain for three days. The pain is moderate in intensity and is located in the lower abdomen. There are no other symptoms such as fever, vomiting, or diarrhea. The patient has had similar episodes in the past year but no specific diagnosis was made. During 

In [51]:
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = azure_client.embeddings.create(
        input=text, model="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    time.sleep(0.5) # rest period to avoid rate limiting on AOAI for free tier
    return embeddings

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 69, Finished, Available)

In [55]:
response = azure_client.embeddings.create(
    input = summarized,
    model= "text-embedding-ada-002"
)

print(response.model_dump_json(indent=2))
# df3=generate_embeddings(summarized)

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 73, Finished, Available)

{
  "data": [
    {
      "embedding": [
        -0.00466447,
        -0.00032476173,
        0.037595946,
        -0.034513894,
        -0.021408804,
        0.01874703,
        -0.04253742,
        -0.043174207,
        -0.032985605,
        0.008322815,
        0.02294983,
        0.015843278,
        0.00022287571,
        0.013996595,
        0.0010777633,
        0.011856987,
        0.049542084,
        -0.0015696817,
        0.029012049,
        -0.024006898,
        0.006940986,
        -0.004342892,
        -0.0051547964,
        0.0037729673,
        -0.0069218827,
        0.0024834722,
        0.0064060846,
        -0.03189033,
        -0.003362239,
        -0.009838371,
        0.021332389,
        -0.009392619,
        -0.0111374175,
        0.00031958785,
        -0.019371083,
        0.013869236,
        0.010838127,
        -0.009252526,
        0.008864085,
        -0.016174408,
        0.034513894,
        -0.01274849,
        -0.0013547658,
        0.006453844,
    

In [56]:
# Create a Dataframe 
df = pd.DataFrame({
    "Date": "2024",
    "Patient Name": h['PID'][0][5][0][0],
    "DOB": h['PID'][0][7],
    "Original_Notes": h['OBX'][0][5],
    "Translated_Notes": english_translation,
    "Summary": summarized,
    
})

# Display the DataFrame
display(df)

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 74, Finished, Available)

SynapseWidget(Synapse.DataFrame, 544ddc84-4850-42c8-be58-182288dfb28d)

In [57]:
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return azure_client.embeddings.create(input = [text], model=model).data[0].embedding



StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 75, Finished, Available)

In [59]:
df['Vectors'] = df["Summary"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002')) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 77, Finished, Available)

In [61]:
df

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 79, Finished, Available)

,Date,Patient Name,DOB,Original_Notes,Translated_Notes,Summary,Vectors
0,2024,Doe,19900101,Patient presents with a three-day history of n...,The patient is experiencing abdominal pain tha...,The patient has been experiencing abdominal pa...,"[-0.00466447, -0.00032476173, 0.037595946, -0...."


In [67]:
df.columns

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 85, Finished, Available)

Index(['Date', 'Patient Name', 'DOB', 'Original_Notes', 'Translated_Notes',
       'Summary', 'Vectors'],
      dtype='object')

In [71]:
#  Save to Delta table
# Convert Pandas dataframe to spark dataframe 
# spark_session_results = spark.createDataFrame(df.to_dict)
# df.write.mode("overwrite").format("delta").save("Tables/patient_notes_test")
sparkDF=spark.createDataFrame(df)


StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 89, Finished, Available)

In [73]:
sparkDF.columns

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 91, Finished, Available)

['Date',
 'Patient Name',
 'DOB',
 'Original_Notes',
 'Translated_Notes',
 'Summary',
 'Vectors']

In [77]:
! pip install deltalake

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 95, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 129.7 MB/s eta 0:00:0000:0100:01


In [78]:
from deltalake import DeltaTable
from deltalake.writer import write_deltalake



StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 96, Finished, Available)

In [81]:
# sparkDF.write.mode("overwrite").format("delta").save("Tables/patient_notes_test")
# Write pandas dataframe to Delta tabel
write_deltalake("Tables/patient_notes_test", df,mode="append")

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 99, Finished, Available)

# Persist Data to MongoDB vCore

In [83]:

! pip install pymongo
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 101, Finished, Available)

In [84]:
import json
import datetime
import time

import pymongo

from tenacity import retry, wait_random_exponential, stop_after_attempt

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 102, Finished, Available)

In [85]:
COSMOS_MONGO_USER="user"
COSMOS_MONGO_PWD="HelloWorld2023!"
COSMOS_MONGO_SERVER="fabricdemo.mongocluster.cosmos.azure.com"

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 103, Finished, Available)

In [86]:
mongo_conn = "mongodb+srv://"+COSMOS_MONGO_USER+":"+COSMOS_MONGO_PWD+"@"+COSMOS_MONGO_SERVER+"?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"
mongo_client = pymongo.MongoClient(mongo_conn)

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 104, Finished, Available)

In [87]:
# create a database called TutorialDB
db = mongo_client['Hl7_raw']

# Create collection if it doesn't exist
COLLECTION_NAME = "patient"

collection = db[COLLECTION_NAME]

if COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    db.create_collection(COLLECTION_NAME)
    print("Created collection '{}'.\n".format(COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COLLECTION_NAME))

StatementMeta(, 01fd9a67-f25e-42aa-a949-fedf0267cd03, 105, Finished, Available)

Created collection 'patient'.



In [1]:
# Save to MongoBD vCore
data=df.to_dict(orient='records')
collection.insert_many(data)

StatementMeta(, 879d4c1a-d147-42b1-ab6a-71ae3ceac7db, 3, Finished, Available)

NameError: name 'df' is not defined